# 자동 클래스를 이용한 토크나이저와 모형의 사용

**두 문장의 의미적 유사도 비교**

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

In [11]:
model.config

BertConfig {
  "_name_or_path": "bert-base-cased-finetuned-mrpc",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

In [12]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [13]:
# 의미적으로 유사한 두 문장을 선언
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"

# 토큰화
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
# 모형으로 결과를 예측
logits = model(**tokens).logits
# 소프트맥스를 이용해 결과값을 클래스에 대한 확률로 변환
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 29%
yes: 71%


In [14]:
target_sequence = "The boy quickly ran across the finish line, seizing yet another victory"
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
logits = model(**tokens).logits
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 95%
yes: 5%


**NLTK 영화리뷰 감정분석 학습**

In [15]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split # sklearn에서 제공하는 split 함수를 사용
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids() # movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] # file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [16]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# cuda를 이용한 GPU 연산이 가능하면 cuda를 사용하고, 아니면 cpu를 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# 모델을 gpu로 옮겨서 연산을 준비
model = model.to(device)

batch_size = 10 # 모형으로 한번에 예측할 데이터의 수
y_pred = [] # 전체 예측결과를 저장

num_batch = len(y_test)//batch_size

for i in range(num_batch):
    inputs = tokenizer(X_test[i*batch_size:(i+1)*batch_size], truncation=True, padding=True, return_tensors="pt")
    # 토큰화 결과를 GPU로 이동
    inputs = inputs.to(device)
    # 모형으로 결과를 예측
    logits = model(**inputs).logits
    # 결과값을 클래스에 대한 확률로 변환
    pred = F.softmax(logits, dim=-1)
    # 예측결과를 CPU로 가져와서 넘파이로 변환한 후, argmax로 확률이 가장 큰 클래스를 선택함
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    # 전체 예측결과에 추가
    y_pred.extend(results.tolist())

# gpu 메모리를 비움
torch.cuda.empty_cache()

score = sum(y_test == np.array(y_pred))/len(y_test)
print("NLTK 영화리뷰 감성분석 정확도:", score)

NLTK 영화리뷰 감성분석 정확도: 0.8425
